# This is a sample Jupyter Notebook

Below is an example of a code cell. 
Put your cursor into the cell and press Shift+Enter to execute it and select the next one, or click 'Run Cell' button.

Press Double Shift to search everywhere for classes, files, tool windows, actions, and settings.

To learn more about Jupyter Notebooks in PyCharm, see [help](https://www.jetbrains.com/help/pycharm/ipython-notebook-support.html).
For an overview of PyCharm, go to Help -> Learn IDE features or refer to [our documentation](https://www.jetbrains.com/help/pycharm/getting-started.html).

In [9]:
import pandas as pd
import os

# --- CONFIGURAZIONE ---
DATA_DIR = '../dataset'
FILE_DATA = os.path.join(DATA_DIR, 'VAERSDATA.csv')
FILE_VAX = os.path.join(DATA_DIR, 'VAERSVAX.csv')
FILE_SYMP = os.path.join(DATA_DIR, 'VAERSSYMPTOMS.csv')
OUTPUT_FILE = os.path.join(DATA_DIR, 'VAERS_MERGED_LOGIC_FIX.csv')

print("1. Caricamento file...")
df_data = pd.read_csv(FILE_DATA, encoding='latin1', low_memory=False)
df_vax = pd.read_csv(FILE_VAX, encoding='latin1', low_memory=False)
df_symp = pd.read_csv(FILE_SYMP, encoding='latin1', low_memory=False)

# --- NORMALIZZAZIONE ID ---
# Fondamentale: convertiamo tutto in stringa per evitare errori di unione
df_data['VAERS_ID'] = df_data['VAERS_ID'].astype(str)
df_vax['VAERS_ID'] = df_vax['VAERS_ID'].astype(str)
df_symp['VAERS_ID'] = df_symp['VAERS_ID'].astype(str)

# --- LOGICA RICHIESTA DALL'UTENTE ---
print("2. Applicazione logica separata (Testo vs Versioni)...")

cols_text = ['SYMPTOM1', 'SYMPTOM2', 'SYMPTOM3', 'SYMPTOM4', 'SYMPTOM5']
cols_version = ['SYMPTOMVERSION1', 'SYMPTOMVERSION2', 'SYMPTOMVERSION3', 'SYMPTOMVERSION4', 'SYMPTOMVERSION5']

# PARTE A: CONTEGGIO (Basato SOLO su SYMPTOMVERSION)
# 1. Contiamo quanti campi versione sono pieni nella singola riga (+1 se presente, 0 no)
df_symp['COUNT_ROW'] = df_symp[cols_version].notna().sum(axis=1)
# 2. Sommiamo per ID (per unire i conteggi se il paziente ha più righe)
df_counts = df_symp.groupby('VAERS_ID')['COUNT_ROW'].sum().reset_index()
df_counts.rename(columns={'COUNT_ROW': 'NUMERO_SINTOMI'}, inplace=True)

# PARTE B: TESTO (Basato SOLO su SYMPTOM)
# 1. Mettiamo tutte le colonne testo in verticale
melted_text = df_symp.melt(
    id_vars=['VAERS_ID'],
    value_vars=cols_text,
    value_name='SINTOMO_TEXT'
).dropna(subset=['SINTOMO_TEXT'])

# 2. Uniamo le stringhe per ID
df_texts = melted_text.groupby('VAERS_ID')['SINTOMO_TEXT'].apply(lambda x: ', '.join(x)).reset_index()
df_texts.rename(columns={'SINTOMO_TEXT': 'LISTA_SINTOMI'}, inplace=True)

# Uniamo Conteggio e Testo in un blocco unico
df_sintomi_final = pd.merge(df_counts, df_texts, on='VAERS_ID', how='outer')

# --- PREPARAZIONE ALTRI DATASET ---
# Deduplica Vaccini (1 riga per paziente)
df_vax_dedup = df_vax.drop_duplicates(subset=['VAERS_ID'], keep='first')

# --- MERGE FINALE ---
print("3. Unione tabelle...")

# Data + Vax
merged_step1 = pd.merge(df_data, df_vax_dedup, on='VAERS_ID', how='inner')

# Unione finale con i Sintomi calcolati
final_df = pd.merge(merged_step1, df_sintomi_final, on='VAERS_ID', how='left')

# Pulizia: chi non ha sintomi avrà 0 e stringa vuota
final_df['NUMERO_SINTOMI'] = final_df['NUMERO_SINTOMI'].fillna(0).astype(int)
final_df['LISTA_SINTOMI'] = final_df['LISTA_SINTOMI'].fillna("")

# --- SALVATAGGIO ---
print(f"4. Salvataggio in {OUTPUT_FILE}...")
final_df.to_csv(OUTPUT_FILE, index=False)

print("-" * 30)
print("OPERAZIONE COMPLETATA.")
print(f"Totale righe: {len(final_df)}")
print(final_df[['VAERS_ID', 'NUMERO_SINTOMI', 'LISTA_SINTOMI']].head())

1. Caricamento file...
2. Applicazione logica separata (Testo vs Versioni)...
3. Unione tabelle...
4. Salvataggio in data/VAERS_MERGED_LOGIC_FIX.csv...
------------------------------
OPERAZIONE COMPLETATA.
Totale righe: 1012894
  VAERS_ID  NUMERO_SINTOMI                                      LISTA_SINTOMI
0   902418               2        Hypoaesthesia, Injection site hypoaesthesia
1   902440               1                                           Headache
2   902446               3                    Erythema, Feeling hot, Flushing
3   902464               5  Dizziness, Electrocardiogram normal, Hyperhidr...
4   902465               7  Dysgeusia, Sensory disturbance, Oral pruritus,...
